# Aula 06 &mdash; Data Scraping com API do Twitter

Renato Vimieiro
rv2 {em} cin.ufpe.br

março 2017

In [1]:
import pandas as pd
import numpy as np

import tweepy

from functools import reduce
from collections import Counter
import json

## Introdução

Nas aulas anteriores vimos como obter dados diretamente de uma página web. Para isso, utilizamos a biblioteca BeautifulSoup. A biblioteca facilitava a busca de tags e, consequentemente, de informações úteis nas páginas. Embora tenhamos que lançar mão desse recurso em muitas ocasiões, alguns sites fornecem APIs para obtermos dados de forma semi-estruturada. Esses dados possuem formato livre, mas possuem indicadores do conteúdo. As representações mais usuais são [XML](https://en.wikipedia.org/wiki/XML) e [JSON](https://en.wikipedia.org/wiki/JSON).

O formato JSON tem se tornado um dos mais usados para troca de dados na internet. Ele tem substituido inclusive XML. Muitos bancos de dados NoSQL, como MongoDB, fornecem suporte direto a dados nesse formato. Em outras palavras, JSON é, talvez, o formato franco de representação de dados para Data Science. Sendo assim, discutiremos rapidamente sua representação.

## JSON

O formato JSON (lê-se como *jay-son*) é essencialmente um dicionário. Ele descreve uma sequência de pares (chave,valor). Objetos no formato JSON são delimitados por { e }. Por sua vez, os pares são separados por vírgulas. As chaves e valores são separados por : (dois-pontos).

As chaves devem ser sempre strings em unicode. Os valores podem ser:

- string
- número
- objeto
- vetor
- true
- false
- null

Os vetores referenciados acima são como vetores/listas em uma linguagem de programação. Eles podem conter uma sequência de um ou mais valores, incluindo outros vetores e objetos. A gramática do formato pode ser conferida em http://www.json.org/json-pt.html.

### Exemplo de um dado no formato JSON

O exemplo a seguir foi retirado da Wikipedia. Ele ilustra como poderiam ser armazenados diversos dados sobre uma pessoa. A escolha das chaves é completamente arbitrária e definida pelo desenvolvedor.

```json
{
  "firstName": "John",
  "lastName": "Smith",
  "isAlive": true,
  "age": 25,
  "address": {
    "streetAddress": "21 2nd Street",
    "city": "New York",
    "state": "NY",
    "postalCode": "10021-3100"
  },
  "phoneNumbers": [
    {
      "type": "home",
      "number": "212 555-1234"
    },
    {
      "type": "office",
      "number": "646 555-4567"
    },
    {
      "type": "mobile",
      "number": "123 456-7890"
    }
  ],
  "children": [],
  "spouse": null
}
```

## API do Twitter

O Twitter oferece aos desenvolvedores uma API para permitir a interação entre o site e aplicativos de terceiros. A API é uma das ricas entre as redes sociais e, por isso, muitas pesquisas comerciais e científicas usam a plataforma.

A API do Twitter se subdivide em outras duas APIs: REST e Streaming. A API de REST permite aos desenvolvedores acessar os perfis dos usuários (públicos) assim como seus seguidores, listas, timeline, além de efetuar buscas por termos ou usuários na plataforma. A API de Streaming, por outro lado, é dedicada ao monitoramento em tempo real dos tweets sendo postados na plataforma.

Além dessas duas interfaces públicas, o Twitter ainda oferece uma terceira interface comercial, Gnip, para acesso a grandes volumes de dados. O Gnip, basicamente, fornece acesso à base de dados (pública) inteira do Twitter. Enquanto a API de Streaming faz amostragem dos tweets em tempo real (não é muito claro como isso é feito), o Gnip fornece acesso a efetivamente todos as mensagens sendo postadas. O mesmo acontece em relação ao REST, que limita, por exemplo, a busca por termos, não retornando todos os resultados. O Gnip, ao contrário, permite efetuar uma busca em toda a base.

## Acesso à API do Twitter

O acesso à API requer a criação de um perfil na plataforma e registro de um aplicativo. A autenticação é feita através do protocolo [OAuth](https://en.wikipedia.org/wiki/OAuth). A ideia básica do protocolo é que um usuário possa delegar acesso a recursos em um determinado site a terceiros sem revelar suas credenciais. Resumindo, o protocolo permite que chaves de acesso sejam geradas pelo usuário; essas chaves são passadas a um terceiro, a quem se deseja permitir acesso aos recursos do usuário; finalmente, a parte a quem o acesso foi delegado pode conectar-se ao sistema usando as chaves que lhe foram enviadas.

#### Registrando um novo aplicativo

Para registrar um aplicativo e gerar novas chaves de acesso, o usuário deve ir até o site https://apps.twitter.com.

![https://apps.twitter.com](images/tela_inicial_apps_twitter.png)

Após logar, a tela inicial mostrará as aplicações autorizadas no momento.

![https://apps.twitter.com](images/apps_twitter_logado.png)

Para criar/autorizar um novo aplicativo, clique no botão **Create New App**. Em seguida, você será redirecionado para uma página onde deverá preencher os dados do aplicativo

![https://apps.twitter.com](images/create_new_app.png)

e aceitar os termos e condições de uso

![https://apps.twitter.com](images/create_new_app_accept_terms_confirm.png)

Pronto! Agora você já resgistrou o aplicativo e poderá gerar as chaves de acesso como a seguir.

#### Obtendo as chaves de acesso

Após registrar o novo aplicativo, você será redirecionado à página de gerenciamento do aplicativo

![https://apps.twitter.com](images/new_app_twitter.png)

Configure as permissões da maneira como for mais conveniente. Em geral, para apenas minerar os dados, basta permitir que o aplicativo possa somente ler os dados. Isso evita, ou diminui as chances, de postagens indevidas caso alguém obtenha suas chaves.

Para gerar as chaves de acesso, vá até a aba **Keys and Access Tokens**.

![https://apps.twitter.com](images/consumer_key_twitter.png)

Inicialmente o aplicativo não está autorizado a logar em nome do usuário. Você verá que foram assinalados um identificador e uma chave de acesso de usuário (*Consumer key and secret*), como mostrado na figura acima. Essas chaves servem para autenticar o usuário, contudo, elas não autenticam o aplicativo, como mostrado na figura abaixo.

![https://apps.twitter.com](images/authorize_app_twitter.png)

Para autorizar o aplicativo, clique no botão **Create my access tokens**. O Twitter então gerará novos identificadores e chaves de acesso para o aplicativo (*Access token and secret*). Esses deverão ser usados em conjunto com as chaves de usuário para se ter acesso à API.

![https://apps.twitter.com](images/access_tokens_twitter.png)

É importante lembrar que esses dois pares de identificadores e chaves nunca devem estar legíveis em seu programa. Eles são como senhas e, portanto, devem ser protegidos.

Agora que já obtivemos as chaves de acesso, podemos iniciar a coleta de dados.

## Acessando a API com Tweepy

Existem diversas bibliotecas de Python para acessar a API do Twitter, como listado na página dos desenvolvedores (https://dev.twitter.com/resources/twitter-libraries). As duas bibliotecas mais usadas (pelo que vi na internet) são [Python-Twitter](https://github.com/bear/python-twitter) e [Tweepy](https://github.com/tweepy/tweepy). No entanto, Tweepy está melhor documentada e possui melhor suporte à API de Streaming. Assim, usaremos Tweepy em nossos exemplos.

#### Conectando-se a API com Tweepy

A biblioteca Tweepy, na verdade, é um wrapper para os serviços listados na API do Twitter. Ela se encarrega dos detalhes de codificação e requisições ao servidor, abstraindo as consultas através de funções e métodos. A biblioteca abstrai inclusive os detalhes de autenticação.

O primeiro passo para coletar os dados é autenticar o usuário e o aplicativo. Isso é feito da seguinte forma:


In [2]:
# As chaves estao armazenadas em um arquivo de configuracoes
# Python possui uma biblioteca para processar esses arquivos (ver configparser)
with open('../configs.txt') as f:
    chaves = json.load(f)
    
# Autentica o usuario na API
auth = tweepy.OAuthHandler(chaves["consumer_key"], chaves["consumer_secret"])

# Autentica o aplicativo
auth.set_access_token(chaves["access_token"], chaves["access_token_secret"])


O objeto `auth` contém a sessão autenticada e deverá ser usado nas conexões com a API.

#### Acesso à REST API

O acesso à REST API é feito através de objetos da classe `tweepy.API`. Os objetos dessa classe possuem métodos que encapsulam os acessos às funcionalidades da API. Assim, devemos instaciar a classe usando a conexão criada anteriormente. É importante respeitar os limites impostos pela API, caso contrário, podemos ser bloqueados. As requisições que ultrapassarem os limites do Twitter resultaram em exceções do tipo `tweepy.RateLimitError`. Tweepy pode controlar automaticamente esses acessos. Podemos estabelecer que as chamadas a determinados métodos devem esperar até que o limite de requisições seja reestabelecido. O limite de requisições varia dependendo do tipo de requisição. O Twitter também faz constantes modificações nesses limites. Então, você sempre deve consultar a documentação da API para se certificar dos limites atuais. Eles podem ser conferidos em https://dev.twitter.com/rest/public/rate-limits.

In [3]:
api = tweepy.API(auth, wait_on_rate_limit=False)

O objeto `api` agora é nossa interface de acesso ao Twitter.

Podemos usá-la para saber quais são os trending topics nos diversos lugares onde essa informação é monitorada.

In [4]:
api.trends_available()

[{'country': '',
  'countryCode': None,
  'name': 'Worldwide',
  'parentid': 0,
  'placeType': {'code': 19, 'name': 'Supername'},
  'url': 'http://where.yahooapis.com/v1/place/1',
  'woeid': 1},
 {'country': 'Canada',
  'countryCode': 'CA',
  'name': 'Winnipeg',
  'parentid': 23424775,
  'placeType': {'code': 7, 'name': 'Town'},
  'url': 'http://where.yahooapis.com/v1/place/2972',
  'woeid': 2972},
 {'country': 'Canada',
  'countryCode': 'CA',
  'name': 'Ottawa',
  'parentid': 23424775,
  'placeType': {'code': 7, 'name': 'Town'},
  'url': 'http://where.yahooapis.com/v1/place/3369',
  'woeid': 3369},
 {'country': 'Canada',
  'countryCode': 'CA',
  'name': 'Quebec',
  'parentid': 23424775,
  'placeType': {'code': 7, 'name': 'Town'},
  'url': 'http://where.yahooapis.com/v1/place/3444',
  'woeid': 3444},
 {'country': 'Canada',
  'countryCode': 'CA',
  'name': 'Montreal',
  'parentid': 23424775,
  'placeType': {'code': 7, 'name': 'Town'},
  'url': 'http://where.yahooapis.com/v1/place/3534',

In [34]:
#trendtopics = api.trends_place(455824)
#trendtopics = api.trends_place(455823)
trendtopics = api.trends_place(23424768)
trendtopics

[{'as_of': '2018-03-22T16:39:37Z',
  'created_at': '2018-03-22T16:32:28Z',
  'locations': [{'name': 'Brazil', 'woeid': 23424768}],
  'trends': [{'name': '#LulaLivre',
    'promoted_content': None,
    'query': '%23LulaLivre',
    'tweet_volume': 21890,
    'url': 'http://twitter.com/search?q=%23LulaLivre'},
   {'name': '#DiaMundialDaÁgua',
    'promoted_content': None,
    'query': '%23DiaMundialDa%C3%81gua',
    'tweet_volume': None,
    'url': 'http://twitter.com/search?q=%23DiaMundialDa%C3%81gua'},
   {'name': '#MamitaCNCOFtLuan',
    'promoted_content': None,
    'query': '%23MamitaCNCOFtLuan',
    'tweet_volume': None,
    'url': 'http://twitter.com/search?q=%23MamitaCNCOFtLuan'},
   {'name': '#QuintaDetremuraSdv',
    'promoted_content': None,
    'query': '%23QuintaDetremuraSdv',
    'tweet_volume': 18810,
    'url': 'http://twitter.com/search?q=%23QuintaDetremuraSdv'},
   {'name': 'Marcelo Miranda',
    'promoted_content': None,
    'query': '%22Marcelo+Miranda%22',
    'tweet_

Como pode ser visto, o resultado é uma lista de objetos no formato JSON (um dicionário). Podemos usar Pandas para criar um data frame com os dados obtidos. Repare que a requisição retorna um único objeto com quatro chaves: data da apuração, data da requisição, local, e os tópicos. Nossa intenção é criar um data frame com os dados dos tópicos. As outras informações são complementares e podem ser replicadas no data frame se necessário após a criação.

In [35]:
dfTrendTopics = pd.DataFrame(trendtopics[0]['trends'])

In [36]:
dfTrendTopics.head(10)

,name,promoted_content,query,tweet_volume,url
0,#LulaLivre,None,%23LulaLivre,21890.0,http://twitter.com/search?q=%23LulaLivre
1,#DiaMundialDaÁgua,None,%23DiaMundialDa%C3%81gua,NaN,http://twitter.com/search?q=%23DiaMundialDa%C3...
2,#MamitaCNCOFtLuan,None,%23MamitaCNCOFtLuan,NaN,http://twitter.com/search?q=%23MamitaCNCOFtLuan
3,#QuintaDetremuraSdv,None,%23QuintaDetremuraSdv,18810.0,http://twitter.com/search?q=%23QuintaDetremuraSdv
4,Marcelo Miranda,None,%22Marcelo+Miranda%22,NaN,http://twitter.com/search?q=%22Marcelo+Miranda%22
5,#HAPPYRENJUNDAY,None,%23HAPPYRENJUNDAY,88883.0,http://twitter.com/search?q=%23HAPPYRENJUNDAY
6,Deadpool 2,None,%22Deadpool+2%22,84822.0,http://twitter.com/search?q=%22Deadpool+2%22
7,Paula Toller,None,%22Paula+Toller%22,NaN,http://twitter.com/search?q=%22Paula+Toller%22
8,Sidão e Jean,None,%22Sid%C3%A3o+e+Jean%22,NaN,http://twitter.com/search?q=%22Sid%C3%A3o+e+Je...
9,Marquinhos Cipriano,None,%22Marquinhos+Cipriano%22,NaN,http://twitter.com/search?q=%22Marquinhos+Cipr...


In [37]:
dfTrendTopics.sort_values(by='tweet_volume',ascending=False).\
                head(10)[['name','tweet_volume']]

,name,tweet_volume
48,#InMyBlood,469178.0
11,Bear,103612.0
5,#HAPPYRENJUNDAY,88883.0
6,Deadpool 2,84822.0
36,#JinAppreciationDay,54058.0
49,#apagao,45731.0
0,#LulaLivre,21890.0
14,Bale,20617.0
3,#QuintaDetremuraSdv,18810.0
22,Liam Payne,16093.0


Podemos também obter a lista de tweets de um usuário

In [39]:
trumpsTweets = api.user_timeline(screen_name='realdonaldtrump')
trumpsTweets[0]

Status(_api=<tweepy.api.API object at 0x105023f60>, _json={'created_at': 'Thu Mar 22 10:40:37 +0000 2018', 'id': 976770619424563200, 'id_str': '976770619424563200', 'text': 'Remember when they were saying, during the campaign, that Donald Trump is giving great speeches and drawing big cro… https://t.co/tmNjLRpLXI', 'truncated': True, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/tmNjLRpLXI', 'expanded_url': 'https://twitter.com/i/web/status/976770619424563200', 'display_url': 'twitter.com/i/web/status/9…', 'indices': [117, 140]}]}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 25073877, 'id_str': '25073877', 'name': 'Donald J. Trump', 'screen_name': 'realDonaldTrump', 'location': 'Washington, DC', 'description': '4

O resultado da chamada é uma lista de objetos da classe `Status`. Inspecionando esse objeto, vemos que ele possui um atributo `_json` que armazena o tweet como um dicionário (JSON).

In [40]:
print(type(trumpsTweets[0]._json))
trumpsTweets[0]._json

<class 'dict'>


{'contributors': None,
 'coordinates': None,
 'created_at': 'Thu Mar 22 10:40:37 +0000 2018',
 'entities': {'hashtags': [],
  'symbols': [],
  'urls': [{'display_url': 'twitter.com/i/web/status/9…',
    'expanded_url': 'https://twitter.com/i/web/status/976770619424563200',
    'indices': [117, 140],
    'url': 'https://t.co/tmNjLRpLXI'}],
  'user_mentions': []},
 'favorite_count': 55758,
 'favorited': False,
 'geo': None,
 'id': 976770619424563200,
 'id_str': '976770619424563200',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'en',
 'place': None,
 'retweet_count': 12812,
 'retweeted': False,
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'text': 'Remember when they were saying, during the campaign, that Donald Trump is giving great speeches and drawing big cro… https://t.co/tmNjLRp

Essa representação reflete o modelo de objetos estabelecido pela API do Twitter. Um tweet possui vários campos, como descrito em https://dev.twitter.com/overview/api/tweets. Nem todos os campos são interessantes para a análise de dados. Por exemplo, o tweet contém informações detalhadas sobre o usuário, como o link para a imagem de seu perfil, o link para a imagem que ele usa de fundo, entre outras coisas. Geralmente, esses dados não são úteis para análise. Assim, devemos ser cuidadosos para criar um data frame somente com os dados que julgarmos importantes.

Por exemplo, podemos selecionar os seguintes campos:

- id
- created_at
- entities (hashtags, urls, user_mentions)
- favorite_count
- in_reply_to_screen_name
- in_reply_to_status_id
- in_reply_to_user_id
- lang
- place (country, name)
- retweet_count
- source
- text
- truncated
- user (screen_name, id)

In [11]:
def filtra_campos(tweet):    
    dic = {a[0] : a[1] for a in tweet.items() if a[0] in ['id',
                                    'created_at',
                                    'favorite_count',
                                    'in_reply_to_screen_name',
                                    'in_reply_to_status_id',
                                    'in_reply_to_user_id',
                                    'lang',
                                    'retweet_count',
                                    'source',
                                    'text',
                                    'truncated']}
    dic.update(tweet['entities'])
    if tweet['place']:
        dic.update({'place_'+a[0] : a[1] for a in tweet['place'].items() \
                    if a[0] in ['country', 'name']})
    dic.update({'user_'+a[0] : a[1] for a in tweet['user'].items() \
                if a[0] in ['screen_name', 'id']})
    return dic
    

In [41]:
filtra_campos(trumpsTweets[0]._json)

{'created_at': 'Thu Mar 22 10:40:37 +0000 2018',
 'favorite_count': 55758,
 'hashtags': [],
 'id': 976770619424563200,
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_user_id': None,
 'lang': 'en',
 'retweet_count': 12812,
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'symbols': [],
 'text': 'Remember when they were saying, during the campaign, that Donald Trump is giving great speeches and drawing big cro… https://t.co/tmNjLRpLXI',
 'truncated': True,
 'urls': [{'display_url': 'twitter.com/i/web/status/9…',
   'expanded_url': 'https://twitter.com/i/web/status/976770619424563200',
   'indices': [117, 140],
   'url': 'https://t.co/tmNjLRpLXI'}],
 'user_id': 25073877,
 'user_mentions': [],
 'user_screen_name': 'realDonaldTrump'}

Podemos aplicar a função `filtra_campos` aos tweets recuperados para extrair os campos desejados.

In [42]:
dfTrumpsTweets = pd.DataFrame(list(map(lambda x: filtra_campos(x._json),trumpsTweets)))
dfTrumpsTweets.head()

,created_at,favorite_count,hashtags,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,retweet_count,source,symbols,text,truncated,urls,user_id,user_mentions,user_screen_name
0,Thu Mar 22 10:40:37 +0000 2018,55758,[],976770619424563200,None,NaN,NaN,en,12812,"<a href=""http://twitter.com/download/iphone"" r...",[],"Remember when they were saying, during the cam...",True,"[{'url': 'https://t.co/tmNjLRpLXI', 'expanded_...",25073877,[],realDonaldTrump
1,Thu Mar 22 10:19:57 +0000 2018,139924,[],976765417908776963,None,NaN,NaN,en,44581,"<a href=""http://twitter.com/download/iphone"" r...",[],Crazy Joe Biden is trying to act like a tough ...,True,"[{'url': 'https://t.co/4e33ZxnAw7', 'expanded_...",25073877,[],realDonaldTrump
2,Thu Mar 22 03:04:47 +0000 2018,59226,[],976655903729610752,None,NaN,NaN,en,13918,"<a href=""http://twitter.com/download/iphone"" r...",[],Democrats refused to take care of DACA. Would ...,False,[],25073877,[],realDonaldTrump
3,Thu Mar 22 03:00:36 +0000 2018,74463,[],976654851684945920,None,NaN,NaN,en,17805,"<a href=""http://twitter.com/download/iphone"" r...",[],Got $1.6 Billion to start Wall on Southern Bor...,True,"[{'url': 'https://t.co/0uCYCqGbgf', 'expanded_...",25073877,[],realDonaldTrump
4,Wed Mar 21 19:05:40 +0000 2018,73442,[],976535330835836929,None,NaN,NaN,en,18038,"<a href=""http://twitter.com/download/iphone"" r...",[],.....They can help solve problems with North K...,True,"[{'url': 'https://t.co/hrZ6vrJjVC', 'expanded_...",25073877,[],realDonaldTrump


#### Pesquisando tweets

A API do Twitter fornece ainda acesso à pesquisa de tweets usando termos ou *hashtags*.
A descrição completa de como elaborar termos de busca pode ser encontrada em https://dev.twitter.com/rest/public/search. O acesso é fornecido em Tweepy pelo método [`search`](http://docs.tweepy.org/en/v3.5.0/api.html#API.search) da classe API.

A documentação de Tweepy traz a seguinte descrição

```python
API.search(q[, lang][, locale][, rpp][, page][, since_id][, geocode][, show_user])
```

Contudo, os parâmetros descritos estão desatualizados. A documentação do código traz a lista atual de parâmetros, conforme descrição da API do Twitter. São eles:

```python
'q', 'lang', 'locale', 'since_id', 'geocode',
             'max_id', 'since', 'until', 'result_type', 'count',
              'include_entities', 'from', 'to', 'source'
```

Verifique https://dev.twitter.com/rest/reference/get/search/tweets para ver a descrição de cada um dos parâmetros.


In [43]:
tweetsDataScience = api.search('#datascience',count=100,result_type="recent")
len(tweetsDataScience)

96

In [44]:
tweetsDataScience

[Status(_api=<tweepy.api.API object at 0x105023f60>, _json={'created_at': 'Thu Mar 22 17:21:08 +0000 2018', 'id': 976871409933848583, 'id_str': '976871409933848583', 'text': 'RT @vinod1975: Top 20 Hot #Python #AI and #MachineLearning #OpenSource #Projects \n\n#Intelligence #DataScience #TensorFlow #PyTorch #InsoSec…', 'truncated': False, 'entities': {'hashtags': [{'text': 'Python', 'indices': [26, 33]}, {'text': 'AI', 'indices': [34, 37]}, {'text': 'MachineLearning', 'indices': [42, 58]}, {'text': 'OpenSource', 'indices': [59, 70]}, {'text': 'Projects', 'indices': [71, 80]}, {'text': 'Intelligence', 'indices': [83, 96]}, {'text': 'DataScience', 'indices': [97, 109]}, {'text': 'TensorFlow', 'indices': [110, 121]}, {'text': 'PyTorch', 'indices': [122, 130]}, {'text': 'InsoSec', 'indices': [131, 139]}], 'symbols': [], 'user_mentions': [{'screen_name': 'vinod1975', 'name': 'Vinod Sharma✨', 'id': 90590619, 'id_str': '90590619', 'indices': [3, 13]}], 'urls': []}, 'metadata': {'iso_language_c

O resultado novamente é uma lista de objetos do tipo `Status`, a qual podemos processar da mesma forma como fizemos com os tweets do Trump.

In [45]:
dfTweetsDataScience = pd.DataFrame(list(map(lambda x: filtra_campos(x._json),
                                            tweetsDataScience)))
dfTweetsDataScience.head()

,created_at,favorite_count,hashtags,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,retweet_count,source,symbols,text,truncated,urls,user_id,user_mentions,user_screen_name
0,Thu Mar 22 17:21:08 +0000 2018,0,"[{'text': 'Python', 'indices': [26, 33]}, {'te...",976871409933848583,None,None,None,en,3,"<a href=""https://www.google.com"" rel=""nofollow...",[],RT @vinod1975: Top 20 Hot #Python #AI and #Mac...,False,[],535783927,"[{'screen_name': 'vinod1975', 'name': 'Vinod S...",Tsundu_Mak
1,Thu Mar 22 17:21:07 +0000 2018,0,"[{'text': 'MilanoDigitalWeek', 'indices': [110...",976871408985833472,None,None,None,en,2,"<a href=""http://twitter.com/download/android"" ...",[],RT @MicheleCiavotta: Our friend Olga from @mea...,False,[],246756973,"[{'screen_name': 'MicheleCiavotta', 'name': 'M...",andreamaurino
2,Thu Mar 22 17:21:01 +0000 2018,0,[],976871379923587073,None,None,None,en,0,"<a href=""http://sproutsocial.com"" rel=""nofollo...",[],Did you know that DU members @jumping_uk organ...,True,"[{'url': 'https://t.co/zlvtOnt7o9', 'expanded_...",18159487,"[{'screen_name': 'jumping_uk', 'name': 'Jumpin...",digiunion
3,Thu Mar 22 17:20:58 +0000 2018,0,"[{'text': 'Scottsdale', 'indices': [4, 15]}, {...",976871369475612673,None,None,None,en,0,"<a href=""http://convey.pro"" rel=""nofollow"">Con...",[],Our #Scottsdale office is now #hiring for #Cal...,True,"[{'url': 'https://t.co/sjG9zbZP48', 'expanded_...",16881249,[],thefreeguru
4,Thu Mar 22 17:20:54 +0000 2018,0,"[{'text': 'DataScience', 'indices': [53, 65]}]",976871354309009409,None,None,None,en,6,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",[],RT @AgeroNews: Want to know how to build first...,False,[],4786908621,"[{'screen_name': 'AgeroNews', 'name': 'Agero, ...",promotednow


Agora podemos iniciar nosso processamento. Podemos, por exemplo, identificar as mensagens mais interessantes, considerando o número de retweets.

In [46]:
dfTweetsDataScience.retweet_count.describe()

count     96.000000
mean      14.375000
std       28.304454
min        0.000000
25%        1.000000
50%        3.000000
75%       12.000000
max      111.000000
Name: retweet_count, dtype: float64

In [47]:
[a for a in dfTweetsDataScience[\
    dfTweetsDataScience.retweet_count >= 12].sort_values(by='retweet_count')['text']]

['RT @IainLJBrown: GOP Rep. Wants Commission To Study Artificial Intelligence https://t.co/uyGGZjaNxQ #DataScience #MachineLearning #DeepLear…',
 "RT @Orcanintell: Welcome to the world's first fully #robotic kitchen. \n\n#AI #Robots #Robotics #Food #BigData #ML #DL #IoT #IIoT #MachineLea…",
 'RT @IainLJBrown: GOP Rep. Wants Commission To Study Artificial Intelligence https://t.co/uyGGZjaNxQ #DataScience #MachineLearning #DeepLear…',
 'RT @IainLJBrown: GOP Rep. Wants Commission To Study Artificial Intelligence https://t.co/uyGGZjaNxQ #DataScience #MachineLearning #DeepLear…',
 'RT @IainLJBrown: 5 Tips to Overcome Big Data Security Issues #BigData #DataScience #MachineLearning #DeepLearning #NLP #Robots #AI #IoT htt…',
 'RT @IainLJBrown: 5 Tips to Overcome Big Data Security Issues #BigData #DataScience #MachineLearning #DeepLearning #NLP #Robots #AI #IoT htt…',
 'RT @ipfconline1: Is #Government Ready for #BigData?\n[by @GovLoop RT @pradeeprao_]\n#SmartCity #DataScience #Dataviz #AI #ML #

Observamos que muitas mensagens se tratam de retweets extra-oficiais (não executadas pelo botão de retweet). Essas mensagens citam diversas usuários. Dessa forma, podemos os tweets para encontrar quais são os usuários mais influentes em data science.

Se verificarmos a descrição do objeto Tweet, veremos que ele possui um atributo user_mentions no campo ***entities***. Esse atributo é preenchido pelo Twitter que entrega uma lista de todos os usuários (objetos do tipo User) contidos no texto. Para o nosso exemplo, podemos nos restringir somente ao login do usuário no perfil (screen_name). Depois, podemos usar o screen_name dos usuários mais influentes para recuperar as informações de seus perfis diretamente da API. 

In [48]:
usuariosInfluentes = dfTweetsDataScience.user_mentions.apply(lambda x: 
                                        [y['screen_name'] for y in x])
usuariosInfluentes

0                                           [vinod1975]
1                [MicheleCiavotta, measurence, EwShopp]
2                                          [jumping_uk]
3                                                    []
4                                           [AgeroNews]
5                                           [vinod1975]
6                              [thedatainc, thedatainc]
7                                        [DeepLearn007]
8                                         [bigdataconf]
9                                         [IainLJBrown]
10                                [mitgovlab, DataKind]
11                                          [vinod1975]
12                      [Science_Mktg, Analytic_Clinic]
13                            [Science_Mktg, eking_pgh]
14                   [Science_Mktg, Strat_AI, Strat_AI]
15                                         [GoforthSci]
16                                       [DeepLearn007]
17                                         [thed

In [49]:
top10Influentes = Counter(reduce(lambda x, y: x+y,
                                 usuariosInfluentes.values)).most_common(10)
top10Influentes

[('IainLJBrown', 17),
 ('Strat_AI', 10),
 ('DeepLearn007', 8),
 ('thedatainc', 4),
 ('Fisher85M', 4),
 ('ipfconline1', 4),
 ('GovLoop', 4),
 ('pradeeprao_', 4),
 ('vinod1975', 3),
 ('Science_Mktg', 3)]

Agora que encontramos os dez usuários mais citados nas mensagens, podemos usar a API para recuperar seus dados de perfil. Essa consulta é feita pela interface `lookup_users`.

In [50]:
descTop10Influentes = api.lookup_users(screen_names=[a[0] for a in top10Influentes])
descTop10Influentes

[User(_api=<tweepy.api.API object at 0x105023f60>, _json={'id': 467513287, 'id_str': '467513287', 'name': 'Iain Brown, PhD', 'screen_name': 'IainLJBrown', 'location': 'London', 'description': 'Lead #DataScientist @SASSoftware | Adjunct Prof @UniSouthampton | Author | PhD | #DataScience #AI #ML #DL #NLP #Risk #IoT #BigData #FinTech | Opinions my own', 'url': 'https://t.co/wo8jxLIetd', 'entities': {'url': {'urls': [{'url': 'https://t.co/wo8jxLIetd', 'expanded_url': 'https://www.linkedin.com/in/iainljbrown/', 'display_url': 'linkedin.com/in/iainljbrown/', 'indices': [0, 23]}]}, 'description': {'urls': []}}, 'protected': False, 'followers_count': 54046, 'friends_count': 52722, 'listed_count': 150, 'created_at': 'Wed Jan 18 15:10:25 +0000 2012', 'favourites_count': 457, 'utc_offset': -25200, 'time_zone': 'Pacific Time (US & Canada)', 'geo_enabled': False, 'verified': False, 'statuses_count': 6562, 'lang': 'en', 'status': {'created_at': 'Thu Mar 22 17:33:22 +0000 2018', 'id': 976874491245744

Os objetos do tipo User possuem diversos campos que não são úteis para nossa análise nesse momento. Assim, antes de criar um data frame com os dados dos usuários, devemos filtrá-los, escolhendo somente aquilo que pode ser interessante. Nesse primeiro momento restringiremos nossa análise ao login, nome, descrição, localização, número de seguidores, o número de pessoas que o usuário segue, e o número de tweets que ele já tuitou.

In [51]:
dfTop10Influentes = pd.DataFrame([{'screen_name':a.screen_name,
                                  'name':a.name,
                                   'description':a.description,
                                   'location':a.location,
                                   'followers_count':a.followers_count,
                                   'friends_count':a.friends_count,
                                   'no_tweets':a.statuses_count
                                  } for a in descTop10Influentes],
                                columns = ['screen_name', 'name', 'description',
                                          'location', 'followers_count', 'friends_count', 
                                           'no_tweets'])
dfTop10Influentes

,screen_name,name,description,location,followers_count,friends_count,no_tweets
0,IainLJBrown,"Iain Brown, PhD",Lead #DataScientist @SASSoftware | Adjunct Pro...,London,54046,52722,6562
1,Strat_AI,Strategic #AI,"Strategic #ArtificialIntelligence, #Analytics,...","Maryland, USA",4423,3887,57
2,DeepLearn007,AI,#ArtificialIntelligence #MachineLearning #Deep...,,96357,93697,29797
3,thedatainc,The Data Incubator,"Free Data Science Fellowship in NYC, SF, DC & ...","New York, USA",6341,4528,2146
4,Fisher85M,Michael Fisher,"Full-Time Analyst, Technology Evangelist, #Cyb...","Vernon, CT",39626,13454,20704
5,ipfconline1,ipfconline,Création de Sites Internet Formation et Consei...,"Marseille, France",93138,86500,108950
6,GovLoop,GovLoop,GovLoop is the knowledge network for #governme...,"Washington, DC",26072,3961,53555
7,pradeeprao_,Pradeep Rao,"Father of an angel, Postgraduate, Experienced ...","Bengaluru, India",21107,16977,3011
8,vinod1975,Vinod Sharma✨,"Chief Technology Officer | Keynote Speaker, #D...",Harare - Zimbabwe,14088,12576,5285
9,Science_Mktg,Marketing Sciences,"Human Computer Interaction (#hci), User Experi...","Baltimore, MD, USA",45576,24961,4747


#### Trabalhando com cursores

A API do Twitter fornece mecanismos para que o usuário itere sobre o conjunto de resultados de uma requisição. Ao invés de retornar todos os resultados de uma única vez, a API utiliza o conceito de paginação através de cursores. 

Os resultados são divididos em páginas. Na primeira requisição, assume que a página solicitada é a primeira, usando para isso o parâmetro cursor com valor -1. Junto com os resultados, a API retorna o cursor (índice de página) anterior e o próximo. Na requisição subsequente, o usuário informa o próximo cursor e a API retorna a página seguinte com os outros resultados. O usuário pode, então, iterar sobre mais resultados que o limite imposto em cada requisição.

A biblioteca Tweepy possui uma interface direta para a paginação usada pelo Twitter. O tutorial http://docs.tweepy.org/en/v3.5.0/cursor_tutorial.html detalha como utilizar os cursores. Veremos um exemplo a seguir onde recuperaremos os 1000 tweets mais recentes contendo #datascience.

In [23]:
tweetsDataScience = [tweet for tweet in tweepy.Cursor(api.search, q='#datascience',\
                                  count=100,result_type="recent").items(1000)]
len(tweetsDataScience)

1000

O resultado da pesquisa continua sendo uma lista de objetos do tipo `Status`, os quais podemos processar da mesma forma que fizemos com a consulta direta.

In [24]:
dfTweetsDataScience = pd.DataFrame(list(map(lambda x: filtra_campos(x._json),
                                            tweetsDataScience)))
dfTweetsDataScience.head()

,created_at,favorite_count,hashtags,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,media,place_country,place_name,retweet_count,source,symbols,text,truncated,urls,user_id,user_mentions,user_screen_name
0,Thu Mar 22 16:01:55 +0000 2018,0,"[{'text': 'DataVisualization', 'indices': [51,...",976851476436979713,None,NaN,NaN,en,NaN,NaN,NaN,1,"<a href=""https://spences10.github.io"" rel=""nof...",[],RT @Pivigo: A Comparative Analysis of Top 6 BI...,False,"[{'url': 'https://t.co/P3mitfeEnV', 'expanded_...",969593261479419905,"[{'screen_name': 'Pivigo', 'name': 'Pivigo', '...",fedora_rapp
1,Thu Mar 22 16:01:49 +0000 2018,0,"[{'text': 'AI', 'indices': [7, 10]}, {'text': ...",976851450751004673,None,NaN,NaN,en,NaN,NaN,NaN,0,"<a href=""http://convey.pro"" rel=""nofollow"">Con...",[],Top 10 #AI #technology Trends in 2018. https:/...,True,"[{'url': 'https://t.co/ZulgmQ5XQV', 'expanded_...",3239019327,"[{'screen_name': 'MikeQuindazzi', 'name': 'Mik...",OpenfieldLive
2,Thu Mar 22 16:01:42 +0000 2018,0,"[{'text': 'DataScience', 'indices': [72, 84]},...",976851423324508160,None,NaN,NaN,en,NaN,NaN,NaN,8,"<a href=""https://spences10.github.io"" rel=""nof...",[],RT @IainLJBrown: Infosys invests $1.5 million ...,False,[],944577597555474432,"[{'screen_name': 'IainLJBrown', 'name': 'Iain ...",michavinogrado1
3,Thu Mar 22 16:01:36 +0000 2018,0,[],976851394392199177,None,NaN,NaN,en,NaN,NaN,NaN,1,"<a href=""http://twitter.com/download/android"" ...",[],RT @IHDPscot: @adepledge @resi_uk used data to...,False,[],722394080,"[{'screen_name': 'IHDPscot', 'name': 'IHDP Sco...",FutureXGlobal
4,Thu Mar 22 16:01:35 +0000 2018,0,"[{'text': 'IBMThink2018', 'indices': [108, 121...",976851393507201025,None,NaN,NaN,en,NaN,NaN,NaN,1,"<a href=""http://www.itknowingness.com"" rel=""no...",[],RT @timothymoran: Huge news - @IBMWatson joins...,False,[],213339721,"[{'screen_name': 'timothymoran', 'name': 'Tim ...",itknowingness


In [25]:
dfTweetsDataScience[['favorite_count','retweet_count']].describe()

,favorite_count,retweet_count
count,1000.000000,1000.000000
mean,0.323000,25.112000
std,1.259467,70.357172
min,0.000000,0.000000
25%,0.000000,1.000000
50%,0.000000,6.000000
75%,0.000000,18.000000
max,15.000000,822.000000


In [26]:
top10Influentes = Counter(reduce(lambda x, y: x+y,
                                 dfTweetsDataScience.user_mentions.apply(lambda x: 
                                        [y['screen_name'] for y in x]).values)).most_common(10)
top10Influentes

[('IainLJBrown', 138),
 ('DeepLearn007', 75),
 ('reach2ratan', 48),
 ('KirkDBorne', 42),
 ('MikeQuindazzi', 37),
 ('PwC', 25),
 ('kdnuggets', 24),
 ('omgitsBenHayes', 21),
 ('IHDPscot', 20),
 ('Ronald_vanLoon', 19)]

In [27]:
dfTop10Influentes = pd.DataFrame([{'screen_name':a.screen_name,
                                  'name':a.name,
                                   'description':a.description,
                                   'location':a.location,
                                   'followers_count':a.followers_count,
                                   'friends_count':a.friends_count,
                                   'no_tweets':a.statuses_count
                                  } \
                                  for a in api.lookup_users(screen_names=[a[0] \
                                                                          for a in top10Influentes])],
                                columns = ['screen_name', 'name', 'description',
                                          'location', 'followers_count', 'friends_count', 
                                           'no_tweets'])
dfTop10Influentes

,screen_name,name,description,location,followers_count,friends_count,no_tweets
0,IainLJBrown,"Iain Brown, PhD",Lead #DataScientist @SASSoftware | Adjunct Pro...,London,54038,52729,6551
1,DeepLearn007,AI,#ArtificialIntelligence #MachineLearning #Deep...,,96347,93698,29797
2,reach2ratan,Ratan Jyoti,"#CyberSecurity Researcher & Leader, #influence...",Bangalore,9060,1165,13726
3,KirkDBorne,Kirk Borne,"The Principal Data Scientist at @BoozAllen, Ph...",Booz Allen Hamilton,190434,42404,83963
4,MikeQuindazzi,Mike Quindazzi ✨,Managing Director @StrategyAnd / @PwC • EC @LA...,"Los Angeles, CA",73636,5170,34403
5,PwC,PwC,Latest news and insights from and about the Pw...,Global,78644,639,8511
6,kdnuggets,KDnuggets,"Covering #AI, #Analytics, #BigData, #DataMinin...","Brookline, MA, USA",109777,407,46364
7,omgitsBenHayes,Ben Hayes,"Solving problems, making big data seem small. 🔍","Pittsburgh, PA",3836,512,188
8,IHDPscot,IHDP Scotland,Innovative Healthcare Delivery Programme | Har...,"Edinburgh, Scotland",532,616,1093
9,Ronald_vanLoon,Ronald van Loon,Helping data driven companies generating value...,#NL. Also follow Linkedin Puls,125521,119855,44526


## Usando a API de Streaming

Como foi dito anteriormente, a API REST não foi desenvolvida para a coleta de dados em tempo real. De acordo com a documentação da API (https://dev.twitter.com/rest/reference/get/search/tweets), o mecanismo de busca não foi desenvolvido para servir como base para coleta exaustiva de tweets. Veja o que diz a documentação

    Please note that Twitter’s search service and, by extension, the Search API is not meant to be an exhaustive source of Tweets. Not all Tweets will be indexed or made available via the search interface.
    
Caso seja importante coletar um grande volume de dados para processá-los depois, deve-se usar a [interface/API de streaming](https://dev.twitter.com/streaming/overview). Nesse caso, uma conexão é mantida com o servidor do Twitter por um tempo mais longo. Novos tweets serão enviados ao usuário à medida em que são postados.

Mais uma vez, Tweepy fornece os mecanismos necessários para coletas da API de streaming. Em suma, o usuário deve criar um *listener* que executará as ações correspondentes quando um evento ocorrer. Esse listerner é passado para Tweepy que se encarrega de conectar com a API do Twitter.

In [99]:
from tweepy.streaming import StreamListener
from tweepy import Stream

A classe abaixo define um exemplo de um listener que exibe as mensagens na saída padrão sempre que um novo dado é enviado.

In [100]:
class StdOutListener(StreamListener):

    def on_data(self, data):
        print(data)
        return True

    def on_error(self, status):
        print(status)


O segundo passo para se coletar dados da API de streaming é instanciar um objeto `Stream` que será responsável pela conexão. Devemos passar para o construtor a sessão autenticada da mesma forma que na API REST, e um listener.

In [101]:
stream = Stream(auth, StdOutListener())

Na maior parte dos casos, nossa intenção é em coletar apenas tweets específicos de um assunto. Assim como na interface de busca, podemos informar os termos que estamos interessados no parâmetro **track**. Uma descrição mais completa dos parâmetros está disponível em https://dev.twitter.com/streaming/overview/request-parameters

In [ ]:
stream.filter(track=['vamosport' ,'vamosanta', 
                     'classicodasmultidoes', 
                     'sportxsantacruz', 'sport recife'])